#Gloss2Keypoint / Text2Keypoint

**Progressive Transformers for End-to-End Sign Language Production** (Ben Saunders, Necati Cihan Camgoz, Richard Bowden - ECCV 2020)  

[paper](https://arxiv.org/pdf/2004.14874.pdf)  
[source code(github)](https://github.com/BenSaunders27/ProgressiveTransformersSLP)  

[transformer 참고 youtube](https://www.youtube.com/watch?v=xhY7m8QVKjo&t=26s)


## What to do

### 0. git clone(완료)

In [ ]:
cd "/content/drive/Shareddrives/투빅스_컨퍼런스_수어"

/content/drive/Shareddrives/투빅스_컨퍼런스_수어


In [ ]:
ls

 미성/   dwnet.ipynb          ProgressiveTransformersSLP/  'txt2key result'/
 data/  '풀리퀘 정복기.pdf'   slp_transformer.ipynb


In [ ]:
#!git clone https://github.com/BenSaunders27/ProgressiveTransformersSLP.git #이미 클론 완료

fatal: destination path 'ProgressiveTransformersSLP' already exists and is not an empty directory.


### 1. input data 형식 통일(txt) - 형태소, 키포인트
> (깃헙 코드 설명 중)  
Data format should be parallel .txt files for src, trg and files.  
Each line of the .txt file represents a new sequence, in the same order in each file.  
src file should contain a new source input on each line.  
trg file should contain skeleton data, with each line a new sequence, each frame following on from the previous.  
Joint values were divided by 3 to move to the scale of -1 to 1.  
Each joint value should be separated by a space; " ".  
Each frame is partioned using the known trg_size length, which includes all joints (In 2D or 3D) and the counter.  
Files file should contain the name of each sequence on a new line.  

- (완료) 형태소 3개 이상 포함하는 문장 선별 -> (정은 화자10 sentence) 중복 존재, (유경 화자10 sentence) 102개 문장 추출
- (완료) 형태소에서 특수문자(괄호) 제거
- (완료) 키포인트 개수 통일 -> 254개
- (완료) 102개 문장으로 실험용 trial.gloss, trial.skels, trial.files 구축 -> ProgressivTransformerSLP>Data>tmp>raw 에 있음!
- (완료) 정은 업로드 데이터에서 선별 후 합치기, train, dev, split  -> ProgressivTransformerSLP>Data>tmp에 저장 (train 101, dev 11개의 데이터)

In [ ]:
### 공유 드라이브에 업로드된 데이터 중 sentence에서 transformer 학습시킬만한 애들 찾아보기
# 정은이 폴더 화자10에 sent 있음

In [ ]:
cd "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/정은/화자10/sentence/형태소Json"

/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/정은/화자10/sentence/형태소Json


In [ ]:
ls

In [ ]:
import json
import os

In [ ]:
path = "./"
file_list = os.listdir(path) #파일명 리스트 생성
good = []

for file_name in file_list: #하나씩 불러들이면서 형태소 3개 이상인 애들만 good 리스트에 넣기
    text_list = []
    with open(file_name, "r") as json_file:
        json_data = json.load(json_file)
        #print(json.dumps(json_data, ensure_ascii=False, indent=4)) #출력용

    if len(json_data["data"]) >= 3:
        text_list.append(file_name)
        for j in range(len(json_data["data"])):
            text = json_data["data"][j]["attributes"][0]['name']
            text_list.append(text)
        good.append(text_list)

good

[['NIA_SL_SEN0590_REAL10_F_morpheme.json', '급하다', '병원', '가다', '가다'],
 ['NIA_SL_SEN0601_REAL10_F_morpheme.json', '서울', '기차', '내리다', '가다'],
 ['NIA_SL_SEN0600_REAL10_F_morpheme.json', '여기로', '가다', '가다'],
 ['NIA_SL_SEN0602_REAL10_F_morpheme.json', '서울', '기차', '내리다', '가다'],
 ['NIA_SL_SEN0605_REAL10_F_morpheme.json', '서울', '기차', '목적', '가다'],
 ['NIA_SL_SEN0604_REAL10_F_morpheme.json', '서울', '기차', '도착', '가다'],
 ['NIA_SL_SEN0610_REAL10_F_morpheme.json', '서울', '기차', '가다', '내리다'],
 ['NIA_SL_SEN0603_REAL10_F_morpheme.json', '서울', '기차', '내리다', '가다'],
 ['NIA_SL_SEN0612_REAL10_F_morpheme.json', '도로', '교통', '내리다', '가다'],
 ['NIA_SL_SEN0611_REAL10_F_morpheme.json', '도로', '교통', '내리다', '가다'],
 ['NIA_SL_SEN0613_REAL10_F_morpheme.json', '도로', '교통', '내리다', '가다'],
 ['NIA_SL_SEN0609_REAL10_F_morpheme.json', '서울', '기차', '가다', '내리다'],
 ['NIA_SL_SEN0606_REAL10_F_morpheme.json', '서울', '기차', '곳', '가다'],
 ['NIA_SL_SEN0608_REAL10_F_morpheme.json', '서울', '기차', '가다', '내리다'],
 ['NIA_SL_SEN0607_REAL10_F_morpheme.json', '

In [ ]:
print(len(file_list))
print(len(good))

110
86


중복 다수 존재 -> 원인 규명 필요



#### 유경 폴더
- 화자01(sentence) : 키포인트txt, 형태소Json(282-327, 46개)
- 화자10(sentence) : 키포인트txt, 형태소Json(149-268(03),269-334(04), 186개)
- 전처리 과정에서 원본 키포인트 데이터 중 빈 폴더 있는 것 발견 -> 개수 맞춰서 올림, 다른 사람 폴더도 확인해봐야할듯!

In [ ]:
### input으로 쓸만한 애들 추리기 #화자01
path = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/유경/화자01/sentence/형태소Json/"
file_list = os.listdir(path) #파일명 리스트 생성
good = []

for file_name in file_list: #하나씩 불러들이면서 형태소 3개 이상인 애들만 good 리스트에 넣기
    text_list = []
    with open(path+file_name, "r") as json_file:
        json_data = json.load(json_file)
        #print(json.dumps(json_data, ensure_ascii=False, indent=4)) #출력용

    if len(json_data["data"]) >= 3:
        text_list.append(file_name)
        for j in range(len(json_data["data"])):
            text = json_data["data"][j]["attributes"][0]['name']
            text_list.append(text)
        good.append(text_list)

good

[['NIA_SL_SEN0283_REAL01_F_morpheme.json', '지하철', '반대', '건너', '가다'],
 ['NIA_SL_SEN0297_REAL01_F_morpheme.json',
  '지하철',
  '때',
  '지나가다',
  '네모',
  '증명',
  '필요'],
 ['NIA_SL_SEN0323_REAL01_F_morpheme.json', '물건', '잃어버리다', '점검', '두다', '곳'],
 ['NIA_SL_SEN0320_REAL01_F_morpheme.json', '약', '지하철', '3호', '바꾸다', '어떻게'],
 ['NIA_SL_SEN0314_REAL01_F_morpheme.json', '보다', '번호', '필요'],
 ['NIA_SL_SEN0302_REAL01_F_morpheme.json', '아기', '혼자', '돌아다니다'],
 ['NIA_SL_SEN0318_REAL01_F_morpheme.json', '천호', '지하철', '8호', '바꾸다', '어떻게'],
 ['NIA_SL_SEN0306_REAL01_F_morpheme.json', '교통', '카드', '충천', '되다'],
 ['NIA_SL_SEN0327_REAL01_F_morpheme.json', '지하철', '곳', '24', '있다'],
 ['NIA_SL_SEN0324_REAL01_F_morpheme.json', '지하철', '곳', '카페', '있다'],
 ['NIA_SL_SEN0317_REAL01_F_morpheme.json', '천호', '지하철', '8호', '바꾸다'],
 ['NIA_SL_SEN0313_REAL01_F_morpheme.json', '보다', '번호', '받다'],
 ['NIA_SL_SEN0292_REAL01_F_morpheme.json', '지하철', '환풍구', '물건', '빠지다'],
 ['NIA_SL_SEN0291_REAL01_F_morpheme.json', '지하철', '환풍구', '가방', '빠지다'],
 ['

화자01에 있는게 화자10에 다 포함되어있음(동일문장 다른화자)

In [ ]:
### input으로 쓸만한 애들 추리기 #화자10 sentence
path = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/유경/화자10/sentence/형태소Json/"
file_list = os.listdir(path) #파일명 리스트 생성
good = []

for file_name in file_list: #하나씩 불러들이면서 형태소 3개 이상인 애들만 good 리스트에 넣기
    text_list = []
    with open(path+file_name, "r") as json_file:
        json_data = json.load(json_file)
        #print(json.dumps(json_data, ensure_ascii=False, indent=4)) #출력용

    if len(json_data["data"]) >= 3:
        text_list.append(file_name)
        for j in range(len(json_data["data"])):
            text = json_data["data"][j]["attributes"][0]['name']
            text_list.append(text)
        good.append(text_list)

good

[['NIA_SL_SEN0162_REAL10_F_morpheme.json', '도착', '알다', '주다'],
 ['NIA_SL_SEN0199_REAL10_F_morpheme.json', '택시', '문자(앱)', '나'],
 ['NIA_SL_SEN0180_REAL10_F_morpheme.json', '한달', '내역', '보다'],
 ['NIA_SL_SEN0165_REAL10_F_morpheme.json', '지하철', '계단', '번호'],
 ['NIA_SL_SEN0179_REAL10_F_morpheme.json', '한달', '내역', '보다'],
 ['NIA_SL_SEN0151_REAL10_F_morpheme.json', '물건', '짐', '차', '드렁크', '옮기다'],
 ['NIA_SL_SEN0166_REAL10_F_morpheme.json', '지하철', '위', '곳'],
 ['NIA_SL_SEN0205_REAL10_F_morpheme.json', '마포(복숭아)', '다리', '가다(차)'],
 ['NIA_SL_SEN0152_REAL10_F_morpheme.json', '물건', '짐', '차', '드렁크', '옮기다'],
 ['NIA_SL_SEN0161_REAL10_F_morpheme.json', '도착', '알다', '주다'],
 ['NIA_SL_SEN0164_REAL10_F_morpheme.json', '핸드폰', '번호', '필요'],
 ['NIA_SL_SEN0207_REAL10_F_morpheme.json', '잠실', '다리', '가다'],
 ['NIA_SL_SEN0206_REAL10_F_morpheme.json', '마포(복숭아)', '다리', '가다(차)'],
 ['NIA_SL_SEN0265_REAL10_F_morpheme.json', '도착', '할수있다', '몇분'],
 ['NIA_SL_SEN0216_REAL10_F_morpheme.json', '학교', '보다', '내리다'],
 ['NIA_SL_SEN0214_REAL10

In [ ]:
print(len(file_list))
print(len(good))

186
102


화자10 102개로 trial.gloss, trial.skels, trial.files 구축

In [ ]:
## 1. trial.gloss(src)  #형태소 txt 파일 생성
file = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/유경/화자10/sentence/trial.gloss"

with open(file, 'w') as f:
    for i in range(len(good)):
        for j in good[i][1:]:
            if "(" in j:
                #targ = good[1][1:][1]
                rem = j.split("(")
                f.write(rem[0]+" ")
                if j == good[i][-1]:
                    f.write(rem[1].rstrip(")"))
                else:
                    f.write(rem[1].rstrip(")")+" ")

            else:
                if j == good[i][-1]:
                    f.write(j)
                else:
                    f.write(j+" ")
        #f.writelines(good[i][1:])
        f.write("\n")


In [ ]:
#check
r_file = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/유경/화자10/sentence/trial.gloss"
with open(r_file, 'r') as fr:
    data = fr.readlines()
print(data)

['도착 알다 주다\n', '택시 문자 앱 나\n', '한달 내역 보다\n', '지하철 계단 번호\n', '한달 내역 보다\n', '물건 짐 차 드렁크 옮기다\n', '지하철 위 곳\n', '마포 복숭아 다리 가다 차\n', '물건 짐 차 드렁크 옮기다\n', '도착 알다 주다\n', '핸드폰 번호 필요\n', '잠실 다리 가다\n', '마포 복숭아 다리 가다 차\n', '도착 할수있다 몇분\n', '학교 보다 내리다\n', '아이 집 보다 내리다\n', '잠실 다리 가다\n', '약 보다 내리다\n', '계산 빼다 받다 돈\n', '신호등 건너 내리다\n', '지하철 계단 곳 12\n', '강남 가다 목적 지하철 바꾸다 몇호\n', '도로 쥐 내리다\n', '신호등 곳 내리다\n', '신호등 쥐 내리다\n', '지하철 계단 12 어느\n', '롯데월드 목적 어느 호선\n', '지하철 마지막 시간\n', '지하철 곳 9호\n', '도로 쥐 내리다\n', '새벽 첫 지하철 시간\n', '낯선 지하철 가깝다 곳\n', '서울 기차 가다 목적 지하철 바꾸다 몇호\n', '지하철 몇 분\n', '도로 쥐 내리다\n', '종로가 가다 방법\n', '신호등 곳 내리다\n', '도로 쥐 내리다\n', '계산 빼다 주다 돈\n', '십자로 건너다 신호등 내리다\n', '오늘 하루 수고\n', '아이 집 보다 내리다\n', '서울역 목적 어느 호선\n', '국립 박물관 목적 지하철 호선\n', '학교 보다 내리다\n', '신호등 건너 내리다\n', '십자로 전 버스 곳 가다 내리다\n', '십자로 전 내리다\n', '낯선 지하철 주차 되다\n', '신호등 횡단보도 건너\n', '길 지도 보다 찾다 할수있다\n', '천호 지하철 8호 바꾸다 어떻게\n', '아이 귀신 찾다 도움\n', '지하철 속 없다\n', '서울 지하철 1달 표 사다 원하다\n', '고속 지하철 9호 바꾸다 어떻게\n', '어린이집 주차 되다\n', '보다 번호 받다\n', '약 지하철 3호 바꾸다\n', 

In [ ]:
#original
r_file = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/ProgressiveTransformersSLP/Data/tmp/train.gloss"
with open(r_file, 'r') as fr:
    data = fr.readlines()
print(data)

['JETZT WETTER MORGEN DONNERSTAG ZWOELF FEBRUAR\n', 'ORT REGEN DURCH REGEN KOENNEN UEBERSCHWEMMUNG KOENNEN\n', 'NORDWEST HEUTE NACHT TROCKEN BLEIBEN SUEDWEST KOENNEN REGEN ORT GEWITTER DAZU\n', 'TAGSUEBER OFT REGEN GEWITTER KOENNEN MANCHMAL REGEN VIEL REGEN\n', 'WOLKE LOCH SPEZIELL NORDWEST\n']


In [ ]:
## 2. trial.files #키포인트 파일명

file = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/유경/화자10/sentence/trial.files"

with open(file, 'w') as f:
    for i in range(len(good)):
        text = good[i][0].replace("_morpheme.json", "")
        f.writelines(text)
        f.write("\n")


In [ ]:
#check
r_file = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/유경/화자10/sentence/trial.files"
with open(r_file, 'r') as fr:
    data = fr.readlines()
print(data)

['NIA_SL_SEN0162_REAL10_F\n', 'NIA_SL_SEN0199_REAL10_F\n', 'NIA_SL_SEN0180_REAL10_F\n', 'NIA_SL_SEN0165_REAL10_F\n', 'NIA_SL_SEN0179_REAL10_F\n', 'NIA_SL_SEN0151_REAL10_F\n', 'NIA_SL_SEN0166_REAL10_F\n', 'NIA_SL_SEN0205_REAL10_F\n', 'NIA_SL_SEN0152_REAL10_F\n', 'NIA_SL_SEN0161_REAL10_F\n', 'NIA_SL_SEN0164_REAL10_F\n', 'NIA_SL_SEN0207_REAL10_F\n', 'NIA_SL_SEN0206_REAL10_F\n', 'NIA_SL_SEN0265_REAL10_F\n', 'NIA_SL_SEN0216_REAL10_F\n', 'NIA_SL_SEN0214_REAL10_F\n', 'NIA_SL_SEN0208_REAL10_F\n', 'NIA_SL_SEN0211_REAL10_F\n', 'NIA_SL_SEN0259_REAL10_F\n', 'NIA_SL_SEN0223_REAL10_F\n', 'NIA_SL_SEN0242_REAL10_F\n', 'NIA_SL_SEN0240_REAL10_F\n', 'NIA_SL_SEN0235_REAL10_F\n', 'NIA_SL_SEN0226_REAL10_F\n', 'NIA_SL_SEN0232_REAL10_F\n', 'NIA_SL_SEN0243_REAL10_F\n', 'NIA_SL_SEN0237_REAL10_F\n', 'NIA_SL_SEN0250_REAL10_F\n', 'NIA_SL_SEN0246_REAL10_F\n', 'NIA_SL_SEN0234_REAL10_F\n', 'NIA_SL_SEN0251_REAL10_F\n', 'NIA_SL_SEN0266_REAL10_F\n', 'NIA_SL_SEN0241_REAL10_F\n', 'NIA_SL_SEN0252_REAL10_F\n', 'NIA_SL_SEN02

In [ ]:
## 3. trial.skels(trg)  #정자세 프레임 앞 30개(!초), 뒤 30개(1초) 제거

reads = [] #102개 문장 키포인트 txt 파일명 리스트
for i in range(len(good)):
    text = good[i][0].replace("_morpheme.json", "")
    reads.append(text)

w_file = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/유경/화자10/sentence/trial.skels"

with open(w_file, 'w') as fw: #한 줄에 한 문장씩, 한 문장마다 앞뒤 1초씩 제거한 프레임별 키포인트 254개씩 연속으로 붙이기
    for r in reads:
        r_file = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/유경/화자10/sentence/키포인트txt/"+ r +".txt"
        with open(r_file, 'r') as fr:
            data = fr.readlines()

        for i in data[31:-30]: #앞뒤 1초 자르기
            each = i.split(",")
            each[-1] = each[-1].rstrip("\n")
            each = [line.strip(" ") for line in each]
            fw.write(" ".join(each[1:]))
            if i == data[-31]:
                continue
            else:
                fw.write(" ")

        fw.write("\n")

In [ ]:
#check
r_file = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/유경/화자10/sentence/trial.skels"
with open(r_file, 'r') as fr:
    data = fr.readlines()
data[0:5] #너무 길어서 5개만 출력해보기

['0.52149415 0.2295165 0.52297854 0.4065061 0.4555547 0.40921095 0.43405566 0.6246189 0.42336622 0.81522393 0.5935303 0.40384114 0.6165039 0.6190712 0.62417966 0.8098438 0.5215185 0.8398134 0.47237694 0.8425469 0.57208985 0.8371146 0.5076123 0.20770921 0.53833985 0.20756944 0.48771727 0.24842276 0.5582666 0.24849218 0.48670116 0.22113195 0.48709276 0.23714669 0.48865724 0.2517691 0.48983398 0.2663915 0.49257812 0.28171006 0.49727538 0.29354775 0.5047168 0.30190277 0.5121582 0.30817014 0.5215576 0.3102587 0.53096193 0.30886632 0.5399707 0.30468836 0.54819334 0.2956363 0.5525 0.28310332 0.5548535 0.26778385 0.55563474 0.25385764 0.5564209 0.23784287 0.5579834 0.22182813 0.4933594 0.20093837 0.49806154 0.1939757 0.5031494 0.19188716 0.5090283 0.19188716 0.51490235 0.1939757 0.5270459 0.19258334 0.53331053 0.18840538 0.5395752 0.1877092 0.54584473 0.18979774 0.55015135 0.1967613 0.521167 0.2106875 0.5215576 0.22113195 0.52234375 0.22879079 0.523125 0.23853905 0.5164697 0.2517691 0.5195996 

In [ ]:
#original
r_file = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/ProgressiveTransformersSLP/Data/tmp/train.skels"
with open(r_file, 'r') as fr:
    data = fr.readlines()
data[0:5]

['-0.00565 -0.20610 0.00055 0.00000 0.00000 0.00000 0.16743 0.02912 -0.00016 0.25389 0.26071 0.16992 0.27647 0.33896 0.36781 -0.16580 0.03733 0.00006 -0.27147 0.31816 -0.00018 -0.18947 0.26242 0.18945 -0.18716 0.22865 0.18961 -0.18512 0.20045 0.18967 -0.16512 0.16793 0.18987 -0.14415 0.16851 0.18995 -0.12356 0.14980 0.19004 -0.17322 0.14471 0.18974 -0.15877 0.10704 0.18984 -0.15540 0.08461 0.18993 -0.15251 0.06188 0.19013 -0.16310 0.15363 0.18981 -0.14478 0.11906 0.19002 -0.13370 0.10548 0.19008 -0.12615 0.08837 0.19016 -0.14999 0.17275 0.18981 -0.12830 0.14292 0.18990 -0.12013 0.12778 0.18994 -0.11726 0.11131 0.18998 -0.14249 0.18865 0.18971 -0.12106 0.17177 0.18991 -0.10558 0.16194 0.19011 -0.09205 0.15233 0.19031 0.27457 0.30516 0.36784 0.27443 0.27689 0.36788 0.26863 0.23916 0.36793 0.26428 0.21863 0.36796 0.25933 0.19126 0.36800 0.25629 0.22206 0.36794 0.24470 0.18341 0.36814 0.24689 0.16083 0.36817 0.24982 0.13811 0.36837 0.25048 0.23015 0.36793 0.23619 0.19372 0.36814 0.22837 0.

원본 데이터와 형태 동일하게 완성!

### 1-1. input data 재업로드 및 수정

- (완료) sentence 화자10 전처리 다시한 후 업로드
- (확인중) 미성 화자10 키포인트txt 개수 부족
- (진행중) 중복 확인 및 사용 데이터 선별

---------------------------------

***최종 사용 예정 데이터***

**유경 폴더**
- 화자10(sentence) : 키포인트txt, 형태소Json(149-268(03),269-334(04), 186개)

**정은 폴더**
- 화자02(word) : 키포인트txt, 형태소Json(326-413, 624-828, 290개->288개)
- 화자05(word) : 키포인트txt, 형태소Json(1-110, 110개)
- 화자10(sentence) : 키포인트txt, 형태소Json(590-700, 111개->110개)

**미성 폴더**
- 화자10(sentence) : 키포인트txt, 형태소Json(1-60, 60개->15개)

**정은hih 폴더**
- 화자10(sentence) : 키포인트txt, 형태소Json(1-77, 77개->63개)


*미성과 정은hih 화자10 문장 번호가 일치하는데, 둘 다 일부 결측값이 존재함 어떻게 처리할지 생각할 필요가 있음*  
-> 결측값이 몇개인지? 제거하는것이 어떤지?(유경)

---------------------------------

위 작업이 끝난 후,
- files, gloss, skels 파일 생성
- train, test 생성






#### 유경 폴더
- 화자01(sentence) : 키포인트txt, 형태소Json(282-327, 46개)
- 화자10(sentence) : 키포인트txt, 형태소Json(149-268(03),269-334(04), 186개)
- 키포인트txt start - end 제거 필요 (유경 해결)
- 키포인트txt 뒤 one-hot vector 추가 필요 (유경 해결)

In [ ]:
### 화자01 sentence
path = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/유경/화자01/sentence/형태소Json/"
file_list = os.listdir(path) #파일명 리스트 생성
good01 = []

for file_name in file_list: #하나씩 불러들이면서 good 리스트에 넣기
    text_list = []
    with open(path+file_name, "r") as json_file:
        json_data = json.load(json_file)
        #print(json.dumps(json_data, ensure_ascii=False, indent=4)) #출력용

    # text_list.append(file_name)
    for j in range(len(json_data["data"])):
        text = json_data["data"][j]["attributes"][0]['name']
        text_list.append(text)
    good01.append(text_list)

print(len(good01))
good01[:5]

46


[['지하철', '가방', '두다', '잊다'],
 ['지하철', '반대', '건너', '가다'],
 ['지하철', '반대', '건너', '어떻게'],
 ['아이', '귀신', '찾다', '도움'],
 ['아이', '귀신', '찾다', '도움', '할수있다']]

In [ ]:
### 화자10 sentence
path = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/유경/화자10/sentence/형태소Json/"
file_list = os.listdir(path) #파일명 리스트 생성
good10 = []

for file_name in file_list: #하나씩 불러들이면서 good 리스트에 넣기
    text_list = []
    with open(path+file_name, "r") as json_file:
        json_data = json.load(json_file)
        #print(json.dumps(json_data, ensure_ascii=False, indent=4)) #출력용

    # text_list.append(file_name)
    for j in range(len(json_data["data"])):
        text = json_data["data"][j]["attributes"][0]['name']
        text_list.append(text)
    good10.append(text_list)

print(len(good10))
good10[:5]

186


[['물건', '없다'],
 ['물건', '없다'],
 ['물건', '짐', '차', '드렁크', '옮기다'],
 ['물건', '짐', '차', '드렁크', '옮기다'],
 ['뒤', '드렁크']]

In [ ]:
tmp = []
for i in good01: # 화자01 속 문장이 화자10과 모두 동일한지 재확인
    for j in good10:
        if i == j:
            break
    if j == len(good10)-1 :
        tmp.append(i)
tmp # 모두 동일

[]

화자01에 있는게 화자10에 다 포함되어있음(동일문장 다른화자)

#### 정은 폴더
- 화자02(word) : 키포인트txt, 형태소Json(326-413, 624-828, 290개->288개)
- 화자05(word) : 키포인트txt, 형태소Json(1-110, 110개)
- 화자10(sentence) : 키포인트txt, 형태소Json(590-700, 111개->110개)
- 화자10(sentence) 파일 재생성 필요 (정은 해결, local에서 진행)
- 키포인트txt 뒤 one-hot vector 추가 필요 (정은 해결, local에서 진행)

In [ ]:
### 화자02 word
path = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/정은/화자02/word/형태소Json/"
file_list = os.listdir(path) #파일명 리스트 생성
file_list.sort()
good02 = []

for file_name in file_list: #하나씩 불러들이면서 good 리스트에 넣기
    text_list = []
    with open(path+file_name, "r") as json_file:
        json_data = json.load(json_file)
        #print(json.dumps(json_data, ensure_ascii=False, indent=4)) #출력용

    text_list.append(file_name)
    for j in range(len(json_data["data"])):
        text = json_data["data"][j]["attributes"][0]['name']
        text_list.append(text)
    good02.append(text_list)

print(len(good02))
good02[:5]

288


[['NIA_SL_WORD0326_REAL02_F_morpheme.json', '복습'],
 ['NIA_SL_WORD0327_REAL02_F_morpheme.json', '연필'],
 ['NIA_SL_WORD0328_REAL02_F_morpheme.json', '상장'],
 ['NIA_SL_WORD0329_REAL02_F_morpheme.json', '서점'],
 ['NIA_SL_WORD0330_REAL02_F_morpheme.json', '세미나']]

In [ ]:
path = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/정은/화자02/word/형태소Json/"
file_list = os.listdir(path) #파일명 리스트 생성
file_list.sort()

path = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/정은/화자02/word/키포인트Json/"
file_list2 = os.listdir(path) #파일명 리스트 생성
file_list2.sort()

print(len(file_list))
print(len(file_list2))

t1 = []
for file in file_list:
    t1.append(file.split('_')[2])

t2 = []
for file in file_list2:
    t2.append(file.split('_')[2])

print(set(t2)-set(t1))

288
290
{'WORD0805', 'WORD0773'}


개수가 2개 차이 나는데, NIA_SL_WORD773_REAL05_F_morpheme.json / NIA_SL_WORD805_REAL05_F_morpheme.json이 없음 (제외하고 진행)

In [ ]:
### 화자05 word
path = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/정은/화자05/word/형태소Json/"
file_list = os.listdir(path) #파일명 리스트 생성
good05 = []

for file_name in file_list: #하나씩 불러들이면서 good 리스트에 넣기
    text_list = []
    with open(path+file_name, "r") as json_file:
        json_data = json.load(json_file)
        #print(json.dumps(json_data, ensure_ascii=False, indent=4)) #출력용

    # text_list.append(file_name)
    for j in range(len(json_data["data"])):
        text = json_data["data"][j]["attributes"][0]['name']
        text_list.append(text)
    good05.append(text_list)

print(len(good05))
good05[:5]

110


[['생각', '가볍다'], ['건의하다'], ['무시'], ['물', '삼키다'], ['고민']]

In [ ]:
# 번호가 다르기 때문에 다른 단어일 것으로 보이지만, 혹시 모르니 화자05 속 단어가 화자02가 동일한 것만 출력해보기
tmp = []
for i in good05: 
    for j in good02:
        if i == j: # 동일하면
            tmp.append(i)
            break
tmp # 모두 다름

[]

화자02와 화자05 속 단어는 모두 다름 (둘 다 사용 가능!)

In [ ]:
### 화자10 sentence
path = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/정은/화자10/sentence/형태소Json/"
file_list = os.listdir(path) #파일명 리스트 생성
file_list.sort()
good10 = []

for file_name in file_list: #하나씩 불러들이면서 good 리스트에 넣기
    text_list = []
    with open(path+file_name, "r") as json_file:
        json_data = json.load(json_file)
        #print(json.dumps(json_data, ensure_ascii=False, indent=4)) #출력용

    text_list.append(file_name)
    for j in range(len(json_data["data"])):
        text = json_data["data"][j]["attributes"][0]['name']
        text_list.append(text)
    good10.append(text_list)

print(len(good10))
good10[:5]

110


[['NIA_SL_SEN0590_REAL10_F_morpheme.json', '급하다', '병원', '가다', '가다'],
 ['NIA_SL_SEN0591_REAL10_F_morpheme.json', '여기로', '가다'],
 ['NIA_SL_SEN0592_REAL10_F_morpheme.json', '여기로', '가다'],
 ['NIA_SL_SEN0593_REAL10_F_morpheme.json', '여기로', '내리다'],
 ['NIA_SL_SEN0594_REAL10_F_morpheme.json', '여기로', '내리다']]

개수가 하나 부족한데, NIA_SL_SEN0640_REAL10_F_morpheme.json이 없음 (제외하고 진행)

중복 확인 결과, 수어소는 같지만 영상이 다르므로 그대로 사용!

#### 미성 폴더
- 화자10(sentence) : 키포인트txt, 형태소Json(1-60, 60개->15개)
- 화자06(sentence) : 키포인트txt, 형태소Json(1-60, 60개->17개)

In [ ]:
### 화자06 sentence
path = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/SENTENCE_REAL001/화자06/형태소Json/"
file_list = os.listdir(path) #파일명 리스트 생성
file_list.sort()
good06 = []

for file_name in file_list: #하나씩 불러들이면서 good 리스트에 넣기
    text_list = []
    with open(path+file_name, "r") as json_file:
        json_data = json.load(json_file)
        #print(json.dumps(json_data, ensure_ascii=False, indent=4)) #출력용

    # text_list.append(file_name)
    for j in range(len(json_data["data"])):
        text = json_data["data"][j]["attributes"][0]['name']
        text_list.append(text)
    good06.append(text_list)

print(len(good06))
good06[:5]

60


[['왼쪽'], ['왼쪽'], ['오른쪽'], ['오른쪽'], ['여기']]

In [ ]:
### 화자10 sentence
path = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/SENTENCE_REAL001/화자10/형태소Json/"
file_list = os.listdir(path) #파일명 리스트 생성
file_list.sort()
good10 = []

for file_name in file_list: #하나씩 불러들이면서 good 리스트에 넣기
    text_list = []
    with open(path+file_name, "r") as json_file:
        json_data = json.load(json_file)
        #print(json.dumps(json_data, ensure_ascii=False, indent=4)) #출력용

    # text_list.append(file_name)
    for j in range(len(json_data["data"])):
        text = json_data["data"][j]["attributes"][0]['name']
        text_list.append(text)
    good10.append(text_list)

print(len(good10))
good10[:5]

60


[['왼쪽'], ['왼쪽'], ['오른쪽'], ['오른쪽'], ['여기']]

In [ ]:
tmp = []
for i in good06: # 화자06 속 문장이 화자10과 모두 동일한지 재확인
    for j in good10:
        if i == j:
            break
    if j == len(good10)-1 :
        tmp.append(i)
tmp # 모두 동일

[]

In [ ]:
path = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/SENTENCE_REAL001/화자10/형태소Json/"
file_list = os.listdir(path) #파일명 리스트 생성
file_list.sort()

path = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/SENTENCE_REAL001/화자10/키포인트txt/"
file_list2 = os.listdir(path) #파일명 리스트 생성
file_list2.sort()

print(len(file_list))
print(len(file_list2))

t1 = []
for file in file_list:
    t1.append(file.split('_')[2])

t2 = []
for file in file_list2:
    t2.append(file.split('_')[2])

print(set(t2)-set(t1))

60
15
set()


화자06과 화자10의 문장은 모두 동일하므로 화자10만 사용

키포인트txt는 15, 17개만 존재 -> 이유 찾아야 함

#### 정은hih 폴더
- 화자02(word) : 키포인트txt, 형태소Json(1-77, 77개->73개)
- 화자05(word) : 키포인트txt, 형태소Json(1-77, 77개)
- 화자01(sentence) : 키포인트txt, 형태소Json(1-78, 78개->63개)
- 화자10(sentence) : 키포인트txt, 형태소Json(1-77, 77개->63개)
- 키포인트txt 생성 + one-hot 추가 (정은 해결, local에서 진행)

In [ ]:
### 화자02 word
path = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/hackathon_data_3000/REAL(1500)/WORD/화자02/REAL_WORD_morpheme/"
file_list = os.listdir(path) #파일명 리스트 생성
file_list.sort()
good02 = []

for file_name in file_list: #하나씩 불러들이면서 good 리스트에 넣기
    text_list = []
    with open(path+file_name, "r") as json_file:
        json_data = json.load(json_file)
        #print(json.dumps(json_data, ensure_ascii=False, indent=4)) #출력용

    # text_list.append(file_name)
    for j in range(len(json_data["data"])):
        text = json_data["data"][j]["attributes"][0]['name']
        text_list.append(text)
    good02.append(text_list)

print(len(good02))
good02[:5]

73


[['고민'], ['뻔뻔'], ['수어'], ['남아'], ['눈']]

파일이 4개 없음 (제외하고 진행)

In [ ]:
### 화자05 word
path = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/hackathon_data_3000/REAL(1500)/WORD/화자05/REAL_WORD_morpheme/"
file_list = os.listdir(path) #파일명 리스트 생성
file_list.sort()
good05 = []

for file_name in file_list: #하나씩 불러들이면서 good 리스트에 넣기
    text_list = []
    with open(path+file_name, "r") as json_file:
        json_data = json.load(json_file)
        #print(json.dumps(json_data, ensure_ascii=False, indent=4)) #출력용

    # text_list.append(file_name)
    for j in range(len(json_data["data"])):
        text = json_data["data"][j]["attributes"][0]['name']
        text_list.append(text)
    good05.append(text_list)

print(len(good05))
good05[:5]

77


[['생각', '가볍다'], ['고민'], ['무시'], ['공감'], ['남녀']]

화자05와 화자02는 번호는 같지만 값은 다름

*정은 폴더 내 값과 번호가 같아서 같은 값인지 비교 *

In [ ]:
### 정은 - 화자05 word
path = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/정은/화자05/word/형태소Json/"
file_list = os.listdir(path) #파일명 리스트 생성
good05_ = []

for file_name in file_list: #하나씩 불러들이면서 good 리스트에 넣기
    text_list = []
    with open(path+file_name, "r") as json_file:
        json_data = json.load(json_file)
        #print(json.dumps(json_data, ensure_ascii=False, indent=4)) #출력용

    # text_list.append(file_name)
    for j in range(len(json_data["data"])):
        text = json_data["data"][j]["attributes"][0]['name']
        text_list.append(text)
    good05_.append(text_list)

print(len(good05_))
good05_[:5]

110


[['생각', '가볍다'], ['건의하다'], ['무시'], ['물', '삼키다'], ['고민']]

*정은 폴더 내 화자05 word와 정은hih 폴더 화자02&05word가 일치, 따라서 정은hih의 데이터는 사용하지 않음*

In [ ]:
### 화자01 sentence
path = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/hackathon_data_3000/REAL(1500)/SEN/화자01/REAL_SENTENCE_morpheme/"
file_list = os.listdir(path) #파일명 리스트 생성
file_list.sort()
good01 = []

for file_name in file_list: #하나씩 불러들이면서 good 리스트에 넣기
    text_list = []
    with open(path+file_name, "r") as json_file:
        json_data = json.load(json_file)
        #print(json.dumps(json_data, ensure_ascii=False, indent=4)) #출력용

    # text_list.append(file_name)
    for j in range(len(json_data["data"])):
        text = json_data["data"][j]["attributes"][0]['name']
        text_list.append(text)
    good01.append(text_list)

print(len(good01))
good01[:5]

63


[['왼쪽'], ['오른쪽'], ['오른쪽'], ['여기'], ['여기']]

In [ ]:
### 화자10 sentence
path = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/hackathon_data_3000/REAL(1500)/SEN/화자10/REAL_SENTENCE_morpheme/"
file_list = os.listdir(path) #파일명 리스트 생성
file_list.sort()
good10 = []

for file_name in file_list: #하나씩 불러들이면서 good 리스트에 넣기
    text_list = []
    with open(path+file_name, "r") as json_file:
        json_data = json.load(json_file)
        #print(json.dumps(json_data, ensure_ascii=False, indent=4)) #출력용

    text_list.append(file_name)
    for j in range(len(json_data["data"])):
        text = json_data["data"][j]["attributes"][0]['name']
        text_list.append(text)
    good10.append(text_list)

print(len(good10))
good10[:5]

63


[['NIA_SL_SEN0001_REAL10_F_morpheme.json', '왼쪽'],
 ['NIA_SL_SEN0002_REAL10_F_morpheme.json', '왼쪽'],
 ['NIA_SL_SEN0003_REAL10_F_morpheme.json', '오른쪽'],
 ['NIA_SL_SEN0004_REAL10_F_morpheme.json', '오른쪽'],
 ['NIA_SL_SEN0005_REAL10_F_morpheme.json', '여기']]

In [ ]:
path = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/hackathon_data_3000/REAL(1500)/SEN/화자10/REAL_SENTENCE_morpheme/"
file_list = os.listdir(path) #파일명 리스트 생성
file_list.sort()

path = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/hackathon_data_3000/REAL(1500)/SEN/화자10/REAL_SENTENCE_morpheme/"
file_list2 = os.listdir(path) #파일명 리스트 생성
file_list2.sort()

print(len(file_list))
print(len(file_list2))

t1 = []
for file in file_list:
    t1.append(file.split('_')[2])

t2 = []
for file in file_list2:
    t2.append(file.split('_')[2])

print(set(t2)-set(t1))

63
63
set()


화자01, 화자10 문장 전처리 과정에서 일부 원본 데이터에 문제가 있는 것을 발견 (제외하고 진행)

### 2. 모델 학습 준비

공유드라이브 내 **ProgressiveTransformersSLP** 폴더  
</br>

- (완료) 1. argparse 처리
    - 일단 main.py, training.py argparse 모두 주석처리 하고 직접 선언
    - 대부분 설정들 Base.yaml에 저장되어있어 무리 없을듯

- (일단 완료) 2. config 참고해서 설정 바꾸기
    - 기본 config 설정: Configs > [Base.yaml](https://github.com/BenSaunders27/ProgressiveTransformersSLP/blob/master/Configs/Base.yaml) 
        - `trg_size`: 150 -> **254**
        - `epoch` : 20000 -> 200
        - 그 외 하이퍼파라미터들은 학습시켜보고 조정하면 될 듯!  
    - `training.py`, `data.py` : test set 아직 없는 관계로 test 부분 주석 처리

- (진행중-정은) 3. [data.py](https://github.com/BenSaunders27/ProgressiveTransformersSLP/blob/master/data.py) : input output 호환되도록 코드 수정, 경로 설정  
    - test set 아직 없으므로 test 부분은 전부 주석처리(유경)
    - 이 상태로 train 실행해봤을 때 호환이 안돼서 발생하는 문제들이 꽤 있어보임. 디버깅.....홧팅...
        1. 인코딩 에러: `encoding="utf-8"` -> `encoding='ISO-8859-1'` 로 수정(유경-해결완료)
        2.  line 238) `trg_line` 에서 str -> float 과정 중 에러 발생 : 전처리 수정 후 파일 재업로드 완료 & 제대로 데이터 들어가는 것까지 확인 (정은)
- (진행중-유경) 4. [plot_videos.py](https://github.com/BenSaunders27/ProgressiveTransformersSLP/blob/master/plot_videos.py) : 2d 키포인트로 수정
- _그 외에 해야할것들 여기에 계속 업데이트하고 진행상황 check_
- (**아직**) 최종 모델 돌아가는지 check!
</br>

> 참고자료
1. [파이토치 사용법](https://greeksharifa.github.io/pytorch/2018/11/10/pytorch-usage-03-How-to-Use-PyTorch/)  
2. [torchtext 참고문서](https://wikidocs.net/60314)
    - Pytorch 텍스트 전처리 라이브러리
    - `data.py`에 있는 Field나 build_vocab 이해하는데 도움

In [ ]:
cd "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/ProgressiveTransformersSLP"

/content/drive/Shareddrives/투빅스_컨퍼런스_수어/ProgressiveTransformersSLP


In [ ]:
# 1. argparse 처리
### train

# 원본 실행 코드
#python __main__.py train ./Configs/Base.yaml

## (방법1) colab에서는 __main__.py 실행시킬 필요 없이 config_path 직접 선언해주고 train 바로 돌리기

#from training import train, test #training.py
#config_path = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/ProgressiveTransformersSLP/Configs/Base.yaml"
#train(cfg_file=config_path)

## (방법2) __main__.py, training.py  argparse 부분 전부 주석처리하고 돌려보기

!python __main__.py

Traceback (most recent call last):
  File "__main__.py", line 41, in <module>
    main()
  File "__main__.py", line 37, in main
    train(cfg_file=config_path)
  File "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/ProgressiveTransformersSLP/training.py", line 548, in train
    train_data, dev_data, src_vocab, trg_vocab = load_data(cfg=cfg)
  File "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/ProgressiveTransformersSLP/data.py", line 105, in load_data
    lambda x: len(vars(x)['src'])
  File "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/ProgressiveTransformersSLP/data.py", line 247, in __init__
    [src_line, trg_frames, files_line], fields))
  File "/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py", line 52, in fromlist
    setattr(ex, name, field.preprocess(val))
  File "/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py", line 185, in preprocess
    return self.preprocessing(x)
  File "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/ProgressiveTransformersSLP/data.py", line 80, i

In [ ]:
######디버깅########

#data.py 중 class SignProdDataset(data.Dataset) 중 def __init__() 복사해와서 실행해보기

import io

src_path = "./Data/tmp/train.gloss"
trg_path = "./Data/tmp/train.skels"
file_path = "./Data/tmp/train.files"

with io.open(src_path, mode='r',encoding = 'ISO-8859-1') as src_file, \
        io.open(trg_path, mode='r',encoding = 'ISO-8859-1') as trg_file, \
            io.open(file_path, mode='r',encoding = 'ISO-8859-1') as files_file: #gloss, skels, files 오픈

    examples = []
    i = 0
    # For Source, Target and FilePath
    for src_line, trg_line, files_line in zip(src_file, trg_file, files_file):
        i+= 1

        # Strip away the "\n" at the end of the line
        src_line, trg_line, files_line = src_line.strip(), trg_line.strip(), files_line.strip()
        # print(i) ##루프 몇번 도는지 체크

        # Split target into joint coordinate values
        trg_line = trg_line.split(" ")
        if len(trg_line) == 1:
            continue
        # Turn each joint into a float value, with 1e-8 for numerical stability
        trg_line = [(float(joint) + 1e-8) for joint in trg_line]
        # Split up the joints into frames, using trg_size as the amount of coordinates in each frame
        # If using skip frames, this just skips over every Nth frame
        trg_frames = [trg_line[i:i + 254] for i in range(0, len(trg_line), 254*1)] # trg_size, skip_frame은 일단 숫자로 집어 넣음

        # Create a dataset examples out of the Source, Target Frames and FilesPath
        # if src_line != '' and trg_line != '':
        #     examples.append(data.Example.fromlist(
        #         [src_line, trg_frames, files_line], fields))
    if i == 101 :
        print("파일 제대로 읽어옴") # 끝까지 루프 도는지 확인

파일 제대로 읽어옴


*전처리할 때 254개 마다, 즉 프레임 끝단마다 공백 추가해야 하는데 빠진거같아..!*

--> 해결해서 데이터 재업로드 완료 및 파일 제대로 읽어오는 것 까지 확인,,!

In [ ]:
# 2. configs 체크용으로 출력해보기
from helpers import load_config
c_path = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/ProgressiveTransformersSLP/Configs/Base.yaml"
cc = load_config(c_path)
cc #파일에서 cfg와 같음

{'data': {'dev': './Data/tmp/dev',
  'files': 'files',
  'max_sent_length': 300,
  'skip_frames': 1,
  'src': 'gloss',
  'test': './Data/tmp/test',
  'train': './Data/tmp/train',
  'trg': 'skels'},
 'model': {'bias_initializer': 'zeros',
  'decoder': {'dropout': 0.0,
   'embeddings': {'dropout': 0.0, 'embedding_dim': 256},
   'ff_size': 1024,
   'hidden_size': 256,
   'num_heads': 8,
   'num_layers': 2,
   'type': 'transformer'},
  'embed_initializer': 'xavier',
  'encoder': {'dropout': 0.0,
   'embeddings': {'dropout': 0.0, 'embedding_dim': 256},
   'ff_size': 1024,
   'hidden_size': 256,
   'num_heads': 8,
   'num_layers': 2,
   'type': 'transformer'},
  'future_prediction': 0,
  'gaussian_noise': False,
  'initializer': 'xavier',
  'just_count_in': False,
  'noise_rate': 5,
  'trg_size': 254},
 'training': {'batch_size': 64,
  'clip_grad_norm': 5.0,
  'continue': True,
  'decrease_factor': 0.7,
  'early_stopping_metric': 'dtw',
  'epochs': 200,
  'eval_metric': 'dtw',
  'keep_last_c

---

### 무시

In [ ]:
text_list = []
text_list.append(file_name)
for j in range(len(json_data["data"])):
    text = json_data["data"][j]["attributes"][0]['name']
    text_list.append(text)
text_list

['NIA_SL_SEN0699_REAL10_F_morpheme.json', '저기', '아파트', '가다(차)']

In [ ]:
file_name = "/content/drive/Shareddrives/투빅스_컨퍼런스_수어/data/유경/화자10/sentence/형태소Json/NIA_SL_SEN0151_REAL10_F_morpheme.json"
with open(file_name, "r") as json_file:
    json_data = json.load(json_file)
    print(json.dumps(json_data, ensure_ascii=False, indent=4))


{
    "metaData": {
        "url": "https://blackolivevideo.blob.core.windows.net/sign-language/0928_rhkim160/NIA_SL_SEN0151_REAL10_F.mp4",
        "name": "NIA_SL_SEN0151_REAL10_F.mp4",
        "duration": 7.617,
        "exportedOn": "2020/10/06"
    },
    "data": [
        {
            "start": 2.037,
            "end": 2.358,
            "attributes": [
                {
                    "name": "물건"
                }
            ]
        },
        {
            "start": 2.521,
            "end": 2.949,
            "attributes": [
                {
                    "name": "짐"
                }
            ]
        },
        {
            "start": 3.213,
            "end": 3.512,
            "attributes": [
                {
                    "name": "차"
                }
            ]
        },
        {
            "start": 3.605,
            "end": 4.191,
            "attributes": [
                {
                    "name": "드렁크"
                }
            